In [2]:
from pybaseball import statcast_batter
from pybaseball import playerid_lookup
import pandas as pd
from pybaseball import cache
from pyspark.sql import SparkSession

cache.enable()


In [8]:
# Sample list of player names
player_names = ['david ortiz', 'mike trout', 'bryce harper']  # Add more player names as needed

# Initialize an empty DataFrame to store the OPS data
ops_data = pd.DataFrame(columns=['Player', 'Year', 'OPS'])

# Loop through each player name
for player_name in player_names:
    # Lookup player ID using playerid_lookup
    player_info = playerid_lookup(player_name.split()[1], player_name.split()[0])
    
    if not player_info.empty:
        player_id = player_info['key_mlbam'].iloc[0]
        
        # Fetch player data for the desired date range
        player_data = statcast_batter(start_dt='2008-04-01', end_dt='2023-08-13', player_id=player_id)
        
        player_data['game_date'] = pd.to_datetime(player_data['game_date'])
        # Calculate OPS for each year
        yearly_ops = player_data.groupby(player_data['game_date'].dt.year)['woba_value'].mean()
        
        # Append OPS data to the DataFrame
        ops_data = ops_data.append(pd.DataFrame({'Player': [player_name] * len(yearly_ops),
                                                 'Year': yearly_ops.index,
                                                 'OPS': yearly_ops}), ignore_index=True)
    else:
        print(f"Player {player_name} not found.")

# Display the collected OPS data
ops_data

Gathering Player Data


/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_34312/1802613251.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ops_data = ops_data.append(pd.DataFrame({'Player': [player_name] * len(yearly_ops),


Gathering Player Data


/Applications/anaconda3/lib/python3.9/site-packages/pybaseball/utils.py:295: DtypeWarning: Columns (87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(data.text))
/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_34312/1802613251.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ops_data = ops_data.append(pd.DataFrame({'Player': [player_name] * len(yearly_ops),


Gathering Player Data


/var/folders/pn/c8dt84bx2r9fbfxb1qyp16k80000gn/T/ipykernel_34312/1802613251.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ops_data = ops_data.append(pd.DataFrame({'Player': [player_name] * len(yearly_ops),


,Player,Year,OPS
0,david ortiz,2008,0.361589
1,david ortiz,2009,0.342547
2,david ortiz,2010,0.363279
3,david ortiz,2011,0.392039
4,david ortiz,2012,0.398198
5,david ortiz,2013,0.395434
6,david ortiz,2014,0.338941
7,david ortiz,2015,0.371254
8,david ortiz,2016,0.397737
9,mike trout,2010,0.871429


In [36]:
from pybaseball import statcast_pitcher_expected_stats

# Get data for all pitchers with a minimum of 150 plate appearances against in  2019, 2021, 2022, 2023
pitcher_data_2019 = statcast_pitcher_expected_stats(2019, 150)
pitcher_data_2021 = statcast_pitcher_expected_stats(2021, 150)
pitcher_data_2022 = statcast_pitcher_expected_stats(2022, 150)
pitcher_data_2023 = statcast_pitcher_expected_stats(2023, 150)

# Concatenate the DataFrames
pitcher_combined_data = pd.concat([pitcher_data_2019, pitcher_data_2021, pitcher_data_2022, pitcher_data_2023], ignore_index=True)

# Display the column names of the combined DataFrame
pitcher_combined_data

,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,era,xera,era_minus_xera_diff
0,Bauer,Trevor,545333,2019,911,557,0.230,0.223,0.007,0.429,0.402,0.027,0.316,0.310,0.006,4.48,4.23,0.249
1,Lynn,Lance,458681,2019,875,562,0.243,0.222,0.021,0.390,0.372,0.018,0.294,0.283,0.011,3.67,3.48,0.192
2,Gonzales,Marco,594835,2019,866,657,0.264,0.254,0.010,0.422,0.407,0.015,0.311,0.311,0.000,3.99,4.26,-0.270
3,Minor,Mike,501985,2019,863,588,0.244,0.228,0.016,0.395,0.389,0.006,0.301,0.298,0.003,3.59,3.89,-0.304
4,Bieber,Shane,669456,2019,859,554,0.230,0.236,-0.006,0.393,0.418,-0.025,0.280,0.295,-0.015,3.27,3.80,-0.525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,Suter,Brent,608718,2023,216,156,0.219,0.211,0.008,0.301,0.278,0.023,0.259,0.249,0.010,2.52,2.54,-0.024
1116,Gonzales,Marco,594835,2023,215,163,0.282,0.286,-0.004,0.431,0.441,-0.010,0.334,0.348,-0.014,5.22,5.18,0.040
1117,Falter,Bailey,663559,2023,214,170,0.317,0.283,0.034,0.515,0.470,0.045,0.369,0.340,0.029,5.21,4.91,0.304
1118,Liberatore,Matthew,669461,2023,208,155,0.282,0.297,-0.015,0.453,0.509,-0.056,0.349,0.383,-0.034,5.71,6.49,-0.775


In [38]:
# Save the concatenated DataFrame as a CSV file in the "resources" folder
pitcher_combined_data.to_csv("Resources/pitcher_combined_data.csv", index=False)

In [35]:
from pybaseball import statcast_batter_expected_stats

# Get data for all qualified batters in 2019, 2021, 2022, 2023
batter_data_2019 = statcast_batter_expected_stats(2019, 150)
batter_data_2021 = statcast_batter_expected_stats(2021, 150)
batter_data_2022 = statcast_batter_expected_stats(2022, 150)
batter_data_2023 = statcast_batter_expected_stats(2023, 150)

# Concatenate the DataFrames
batter_combined_data = pd.concat([batter_data_2019, batter_data_2021, batter_data_2022, batter_data_2023], ignore_index=True)

# Display the column names of the combined DataFrame
batter_combined_data

,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff
0,Semien,Marcus,543760,2019,747,556,0.285,0.272,0.013,0.522,0.495,0.027,0.373,0.363,0.010
1,Merrifield,Whit,593160,2019,735,559,0.302,0.280,0.022,0.463,0.431,0.032,0.340,0.324,0.016
2,Acuña Jr.,Ronald,660670,2019,715,439,0.280,0.279,0.001,0.518,0.574,-0.056,0.369,0.390,-0.021
3,Villar,Jonathan,542340,2019,714,472,0.274,0.246,0.028,0.453,0.412,0.041,0.335,0.315,0.020
4,Betts,Mookie,605141,2019,706,505,0.295,0.311,-0.016,0.524,0.577,-0.053,0.380,0.411,-0.031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288,Schmitt,Casey,669477,2023,218,152,0.205,0.229,-0.024,0.290,0.327,-0.037,0.244,0.273,-0.029
1289,Wong,Kolten,543939,2023,216,150,0.165,0.210,-0.045,0.227,0.284,-0.057,0.216,0.260,-0.044
1290,Madrigal,Nick,663611,2023,212,177,0.267,0.284,-0.017,0.346,0.348,-0.002,0.301,0.311,-0.010
1291,Arroyo,Christian,624414,2023,206,153,0.241,0.231,0.010,0.369,0.341,0.028,0.275,0.269,0.006


In [37]:
# Save the concatenated DataFrame as a CSV file in the "resources" folder
batter_combined_data.to_csv("Resources/batter_combined_data.csv", index=False)

In [66]:
# Filter the DataFrame for the relevant years (2019, 2021, and 2023)
# Filter the DataFrame for the relevant years (2019, 2021, and 2023)
relevant_years = [2019, 2021, 2022, 2023]
filtered_data = batter_combined_data[batter_combined_data['year'].isin(relevant_years)]

# Group the data by player
grouped_data = filtered_data.groupby('player_id')

# Calculate the average wOBA for each player
average_woba = grouped_data['woba'].mean()

# Merge the average wOBA with the 2023 wOBA for each player
woba_2023 = pd.merge(average_woba, filtered_data[filtered_data['year'] == 2023][['player_id', 'woba']], on='player_id', suffixes=('_avg', '_2023'))

# Merge player names with the result DataFrame
# Merge player names and calculated results
players_off_output = pd.merge(
    filtered_data[['player_id', 'first_name', 'last_name']].drop_duplicates(),
    average_woba,
    left_on='player_id',
    right_index=True
).merge(
    woba_2023,
    on='player_id',
    suffixes=('_avg', '_2023')
)

# Display the result
players_off_output


,player_id,first_name,last_name,woba,woba_avg,woba_2023
0,543760,Marcus,Semien,0.35225,0.35225,0.351
1,593160,Whit,Merrifield,0.31950,0.31950,0.338
2,660670,Ronald,Acuña Jr.,0.38475,0.38475,0.423
3,605141,Mookie,Betts,0.38100,0.38100,0.406
4,645277,Ozzie,Albies,0.33725,0.33725,0.354
...,...,...,...,...,...,...
284,678225,Ji Hwan,Bae,0.27300,0.27300,0.273
285,608671,Travis,Jankowski,0.34300,0.34300,0.343
286,665828,Oswaldo,Cabrera,0.25000,0.25000,0.250
287,672779,Tucupita,Marcano,0.27500,0.27500,0.275


In [65]:
from scipy.stats import zscore

z_scores_woba_avg = zscore(average_woba)
z_score_woba_2023 = zscore(woba_2023)


In [71]:
# Calculate z-scores for woba_avg and woba_2023
players_off_output['zscore_avg_woba'] = zscore(result_with_names['woba_avg'])
players_off_output['zscore_woba_2023'] = zscore(result_with_names['woba_2023'])

# Calculate the difference between z-scores
players_off_output['zscore_difference'] = players_off_output['zscore_woba_2023'] - players_off_output['zscore_avg_woba']

# Display the result
players_off_output

,player_id,first_name,last_name,woba,woba_avg,woba_2023,zscore_avg_woba,zscore_woba_2023,zscore_difference
0,543760,Marcus,Semien,0.35225,0.35225,0.351,0.948755,0.806333,-0.142422
1,593160,Whit,Merrifield,0.31950,0.31950,0.338,-0.173232,0.450102,0.623334
2,660670,Ronald,Acuña Jr.,0.38475,0.38475,0.423,2.062178,2.779309,0.717132
3,605141,Mookie,Betts,0.38100,0.38100,0.406,1.933706,2.313468,0.379762
4,645277,Ozzie,Albies,0.33725,0.33725,0.354,0.434868,0.888541,0.453673
...,...,...,...,...,...,...,...,...,...
284,678225,Ji Hwan,Bae,0.27300,0.27300,0.273,-1.766283,-1.331057,0.435226
285,608671,Travis,Jankowski,0.34300,0.34300,0.343,0.631858,0.587114,-0.044744
286,665828,Oswaldo,Cabrera,0.25000,0.25000,0.250,-2.554243,-1.961313,0.592930
287,672779,Tucupita,Marcano,0.27500,0.27500,0.275,-1.697764,-1.276252,0.421512


In [74]:
# Sort the DataFrame by greatest increase in z-score difference
sorted_output_batting_woba = players_off_output.sort_values(by='zscore_difference', ascending=False)

# Display the sorted result
sorted_output_batting_woba

,player_id,first_name,last_name,woba,woba_avg,woba_2023,zscore_avg_woba,zscore_woba_2023,zscore_difference
23,641355,Cody,Bellinger,0.332000,0.332000,0.393,0.255007,1.957236,1.702229
73,608369,Corey,Seager,0.374750,0.374750,0.439,1.719586,3.217748,1.498162
197,672695,Geraldo,Perdomo,0.298000,0.298000,0.343,-0.909804,0.587114,1.496918
114,656811,Ryan,O'Hearn,0.298333,0.298333,0.343,-0.898384,0.587114,1.485498
176,673490,Ha-Seong,Kim,0.315333,0.315333,0.363,-0.315978,1.135163,1.451141
...,...,...,...,...,...,...,...,...,...
8,547989,José,Abreu,0.334500,0.334500,0.279,0.340655,-1.166642,-1.507298
47,545361,Mike,Trout,0.408000,0.408000,0.370,2.858703,1.326980,-1.531723
53,516782,Starling,Marte,0.337500,0.337500,0.278,0.443433,-1.194045,-1.637478
79,641313,Tim,Anderson,0.321750,0.321750,0.256,-0.096149,-1.796899,-1.700750


In [75]:
# Save the concatenated DataFrame as a CSV file in the "resources" folder
sorted_output_batting_woba.to_csv("Resources/sorted_output_batting_woba.csv", index=False)